In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "mps"  # the device to load the model onto

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype=torch.half,
    # device_map="auto"
)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

/Users/owner/opt/anaconda3/envs/llm-diss/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
prompt = "Tell me succinctly what qwen means."

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [20]:
print(response)

Qwen is a large language model created by Alibaba Cloud, which was released in 2021. It is the first AI language model developed on a commercially available platform and can understand and generate text across multiple languages. Qwen's capabilities include natural language understanding, generation, and translation.


In [14]:
tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]

'A large language model is a type of artificial intelligence that can generate text using natural language in multiple languages, and it uses the principles of machine learning to learn from vast amounts of data.\n\nThe process of generating text involves several steps:\n\n1. Data collection: The first step is collecting data about the language models. This could include text samples from different sources such as books, articles, or social media posts.\n\n2. Training: Once the data has been collected, the next step is to train the model on this data. This involves feeding the model with new text samples and training it on a set of labeled examples that represent the types of texts that the model will eventually produce.\n\n3. Evaluation: After training, the model needs to be evaluated on a set of test data to see how well it performs. This involves comparing the generated text against the actual text produced by the model.\n\n4. Improvement: As the model learns more, it may improve it

In [7]:
print(text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell me succinctly what qwen means.<|im_end|>
<|im_start|>assistant



In [13]:
tokenizer.eos_token

'<|im_end|>'

In [15]:
prompt = """<|im_start|>user
Summarise the following article:
My boyfriend and I are long distance. We have a trip planned this summer which involves me going over to him in the USA. This will be the second time I have actually been with him in person. I am flying from the UK with my mum to the east coast. The original plan was for me to fly over to my boyfriend in the west coast (my parents are holidaying on the east coast) but because my mum was freaking out so much about me going to meet my boyfriend i said we can all road trip there together. I even invited her on the trip with us. I have given her all of our dates so that she can travel around with us.\n\nThe plan was for me to stay on the 4th July and fly back on the 5th. Mum knew this. I told her I had booked a flight back already from the west coast to east coast (where she would pick me up and we would fly back to the UK together). She has gone mad at me because she can't believe I would book a flight when she told me she didn't want me flying on my own. At the time I had booked it she told me she wasn't gonna road trip with us. She knew the trip was happening.......how else was I to get home if I don't fly? \n\nI am fine flying on my own it doesn't bother me at all. I feel like I have done everything I can to make her feel comfortable with this trip and she is just trying to sabotage it. Thoughts??<|im_end|>
<|im_start|>assistant
"""

# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)

response_2 = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]

print(response_2)

<|im_start|>user
Summarise the following article:
My boyfriend and I are long distance. We have a trip planned this summer which involves me going over to him in the USA. This will be the second time I have actually been with him in person. I am flying from the UK with my mum to the east coast. The original plan was for me to fly over to my boyfriend in the west coast (my parents are holidaying on the east coast) but because my mum was freaking out so much about me going to meet my boyfriend i said we can all road trip there together. I even invited her on the trip with us. I have given her all of our dates so that she can travel around with us.

The plan was for me to stay on the 4th July and fly back on the 5th. Mum knew this. I told her I had booked a flight back already from the west coast to east coast (where she would pick me up and we would fly back to the UK together). She has gone mad at me because she can't believe I would book a flight when she told me she didn't want me fly

In [18]:
# test_text = 'asdf <|im_end|>'


response_2[: -len('<|im_end|>')]

"<|im_start|>user\nSummarise the following article:\nMy boyfriend and I are long distance. We have a trip planned this summer which involves me going over to him in the USA. This will be the second time I have actually been with him in person. I am flying from the UK with my mum to the east coast. The original plan was for me to fly over to my boyfriend in the west coast (my parents are holidaying on the east coast) but because my mum was freaking out so much about me going to meet my boyfriend i said we can all road trip there together. I even invited her on the trip with us. I have given her all of our dates so that she can travel around with us.\n\nThe plan was for me to stay on the 4th July and fly back on the 5th. Mum knew this. I told her I had booked a flight back already from the west coast to east coast (where she would pick me up and we would fly back to the UK together). She has gone mad at me because she can't believe I would book a flight when she told me she didn't want m